In [1]:
%load_ext autoreload
%autoreload 2

# Cognitive Architectures

 Cognitive architectures often use production rules for reasoning. Production rules are logical implications in the form *if* $\alpha$, *then* $\beta$; written in logic, $ \alpha \rightarrow \beta$. If the system believes that $\alpha$ is true, then it can conclude that $\beta$ is true using this rule. 
    
We'll implement a production rule system in Python. We first need a representation of the system's *rules* and its *beliefs*. To start, we'll keep things relatively simple and assume each rule consists of exactly one statement implying another statement, where the statements contain no logical connectives. We'll represent this as a two-item tuple, with the first item equal to the antecedent (if part) and the second item equal to the consequent (then part). For instance, the logical rule "If gas meter at zero, then no gas in tank", would be represented as `('gas meter at zero','no gas in tank')`. We'll represent our beliefs as a set of strings in Python. (If you haven't seen the syntax for sets in Python, [check out the documentation here](https://docs.python.org/3/tutorial/datastructures.html#sets). Most syntax is the same as a dictionary with only keys, no values.)

## Forward chaining

We'll start off using a *forward chaining* system, in which the system repeatedly uses the rules to determine if anything new can be added to its beliefs. For instance, if the system only had the belief "gas meter at zero" and the rule "If gas meter at zero, then no gas in tank", it could use forward chaining to add "no gas in tank" as an additional belief. The system is thus able to consider the logical consequences of its beliefs.

<div class="alert alert-success" role="alert">
<h3>Part (a)</h3>
<p>
Create a module `productionrules.py`. Write a function `get_triggered_rule(beliefs, rules)` that finds a rule that is "triggered" by a given set of beliefs. This function should take two parameters: (1) the beliefs (a set of strings), and (2) a list of rules, where each rule is a two-item tuple. Your function should return the first rule that allows the system to add something new to its beliefs. If no such rule exists, the function should return None.
      <p><p>  
        For example, if the rule is "If a, then b" and the belief set contained a and b, then the rule is not triggered, because b is already part of the set of beliefs. If the belief set contained only b, then the rule is not triggered because the antecedent of the rule is not true. However, if the belief set contains only a, then the rule is triggered because this rule would allow us to conclude b, which we did not already know.
        <p><p>
        In the cell below, write some tests demonstrating that your `get_triggered_rule` function works properly.

In [2]:
import productionrules

# Test one rule is triggered
assert (productionrules.get_triggered_rule({"a"}, [("a", "b")]) == ("a", "b"))
# Test no rule is triggered
assert (productionrules.get_triggered_rule({"a"}, [("b", "c")]) == None)
# Test no rule is triggered since the belief already exists
assert (productionrules.get_triggered_rule({"a", "c"}, [("a", "c")]) == None)
# Test the second rule is triggered
assert (productionrules.get_triggered_rule({"a", "c", "e"},
                                           [("a", "c"), ("e", "f")]) == ("e", "f"))
# Test the second rule is triggered with an extra
# rule that can be potentially triggered
assert (productionrules.get_triggered_rule({"a", "c", "e"},
                                           [("a", "c"), ("e", "f"), ("c", "g")]) == ("e", "f"))
# Test no belief situation
assert (productionrules.get_triggered_rule({}, [("a", "c")]) == None)
# Test no rule and no belief situation
assert (productionrules.get_triggered_rule({}, []) == None)
# Test the 'if' condition does not exist but the 'then' condition exists
assert (productionrules.get_triggered_rule({"c"}, [("a", "c")]) == None)

<div class="alert alert-success" role="alert">
<h3>Part (b)</h3>
<p>
In forward chaining, we keep adding to our beliefs by executing the triggered rules until no more rules are triggered. "Executing a rule" simply means adding its consequent to our beliefs. 
<p><p>
Here's an example: We start with the rules "If belief1, then belief2" and "If belief2, then belief3", and the only belief in our belief set is "belief1". We first execute rule 1, increasing our belief set to contain "belief1" and "belief2". Then, with this expanded set of beliefs, rule 2 is trigger. We execute it so that our final set of beliefs is "belief1", "belief2", and "belief3".

<p>
In your module, write a function `forward_chain(beliefs, rules)`, to automatically perform forward chaining using a given set of initial beliefs and list of rules. This function should return a tuple where the first item is the final set of beliefs and the second item is a list of rules that were triggered, in the order they were triggered. This function should use the function you wrote in the previous part, and should not modify the passed in beliefs or rules (i.e., it should be non-destructive).

In [3]:
test_rule = [("a", "d"), ("c", "a"), ("d", "f")]
test_belief = {"c"}
expected_output = ({'d', 'c', 'f', 'a'}, [('c', 'a'), ('a', 'd'), ('d', 'f')])

# Test Case
assert productionrules.forward_chain(test_belief, test_rule) == expected_output
# Test for non-destructive
assert test_rule == [("a", "d"), ("c", "a"), ("d", "f")] and test_belief == {"c"}

#### Forward chaining example 1

 Imagine you have an old car whose battery often dies without warning. When your car battery dies you can't charge your phone, turn on your headlights, or start the car. To make matters worse, you also happen to be quite forgetful and often neglect to fill up the gas tank when it is running low. When your gas tank is empty the gas meter on your dashboard reads zero and your car won't start. We can represent this scenario in a production system as follows:

In [4]:
rules1 = [("gas meter at zero", "no gas in tank"),
          ("no gas in tank", "car won't start"),
          ("car battery is dead", "head lights won't turn on"),
          ("car battery is dead",
          "phone charger doesn't work"),
          ("car battery is dead", "car won't start")]
print(productionrules.forward_chain({"no gas in tank"}, rules1)[0])

{'no gas in tank', "car won't start"}


<div class="alert alert-success" role="alert">
<h3>Part (c)</h3>
<p>
In the cell above (with the rules), add code that uses your function(s) from above to run forward chaining given that you know that the car's gas meter is at zero (and you start off knowing nothing else). In the cell below, write down what you can conclude  about the states of the gas tank, head lights, phone charger, and car.

gas tank: no gas in tank

head lights: N/A

phone charger: N/A

car: car won't start

#### Forward chaining example 2

Now, let's imagine a different production system that is similar but ultimately unrelated to the one we defined in the previous part:

In [5]:
rules2 = [("no gas in tank", "gas meter at zero"),
          ("head lights won't turn on", "car battery is dead"),
          ("my phone charger doesn't work", "car battery is dead"),
          ("car won't start", "car battery is dead")]
print(productionrules.forward_chain({"no gas in tank"}, rules2)[0])

{'no gas in tank', 'gas meter at zero'}


<div class="alert alert-success" role="alert">
<h3>Part (d)</h3>
<p>
In this new production system, if you knew only that your car won't start what would you conclude?  Intuitively, do your conclusion(s) seem like valid or reasonable inferences to make given the rules of your production system? Explain in the cell below. Include code to support your conclusions if appropriate. 

We will conclude that the car battery is dead. This is valid given the all the input in this system. Specifically, the belief "car won't start" will result in "car battery is dead". Whereas "car battery is dead" does not conclude any other beliefs. Thus, ("car won't start", "car battery is dead") is the only rule that is triggered, which is supported by the output of our program. 

However, intuitively, it is hard for human being to make conclusion about the car battery if we only know car won't start. We would be more likely to conclude something related to the tank. Also, it is not that logical to make conclusion about the car battery if we only know that car won't start.

#### Forward chaining example 3

Finally, we can imagine a third production system where the rules apply in both directions:

In [6]:
rules3 = [("gas meter at zero", "no gas in tank"),
          ("no gas in tank", "car won't start"),
          ("car battery is dead", "head lights won't turn on"),
          ("car battery is dead", "phone charger doesn't work"),
          ("car battery is dead", "car won't start"),
          ("no gas in tank", "gas meter at zero"),
          ("head lights won't turn on", "car battery is dead"),
          ("phone charger doesn't work", "car battery is dead"),
          ("car won't start", "car battery is dead")]
print(productionrules.forward_chain({"no gas in tank"}, rules3)[0])

{"head lights won't turn on", 'car battery is dead', 'no gas in tank', "car won't start", "phone charger doesn't work", 'gas meter at zero'}


<div class="alert alert-success" role="alert">
<h3>Part (e)</h3>
<p>
These rules are in "Forward chain example 3". If you observed "no gas in tank", what would you conclude? Does this conclusion seem reasonable given what you know about the meaning of the propositions? Briefly explain your reasoning. Include code to support your conclusions if appropriate.

We will conclude that: 1. A car battery is dead. 2. Car won't start. 3. A gas meter at zero. 4. Headlights won't turn on. 5. Phone charger doesn't work. 
Some of the conclusions are reasonable. The propositions 'no gas in the tank', 'car won't start' and 'gas meter at zero' make perfect sense givn the observation 'no gas in tank'. 
However, the conclusion 'car battery is dead' is not reasonable. Just as what we argue in previous question, 'car won't start' does not necessarily result in 'car battery is dead'. If this conclusion is questionable, the facts that 'headlight won't turn on' and 'phone charger doesn't work' are also questionable. 

### Limits of production rule systems

<div class="alert alert-success" role="alert">
<h3>Part (f)</h3>
<p>
Do the parts above tell us anything about the limits of a basic production rule system for making inferences? In the cell below, explain how the system's conclusions are consistent with or different from the types of conclusions that people make. Your answer should include how you think people would behave given a situation similar to the one in earlier parts of the questions, and should discuss the types of conclusions people might draw given particular evidence.

The conclusion that a basic production rule system provides is largely depended on how the rules are set up. In order to provide enough valid conclusions, input rules need to include all the possible cases and have to be correct. This is different from how people think. When we meet a similar situation, our brain will automatically sort the 'rules', and instantly apply the rule that is related to the given situation based on our experience, knowledge, and intuition. We will not store all the rules in our brain, but the rule will be easy to generate based on what has already stored. For example, given the same situation 'no gas in the tank', we will not loop through all the possible 'rules' in our head. Instead, the fact that 'gas meter at zero' will be derived by logical reasoning based on our understanding about the function of the gas meter. Then, our brain will apply our knowledge of physics or real-world experience and concludes that car battery is dead. 

Also, we only want to generate the conclusion that is useful. Take the same example about the car, we will conclude 'headlight won't turn on' when we need to turn the headlight on. Similarly, we will conclude 'phone charger doesn't work' when we want to use the phone charger. However, the production rule system here will generate all the possible conclusions, which may include a lot of useless information. 

Last but not least, the rule only implies one direction. While in reality, the two statements could both be the 'cause' of one another. Or, the two statements might be two necessary conditions for another statement. However, production rule system does not handle these two problems well. Of course, both problems could be solved by increasing the number of rules and make the lists of the rule include all the possible cases. But this will makes the input very long and redundant.

## Backward chaining

Another type of reasoning used in production rule systems is backward chaining. In backward chaining, we still have beliefs and rules, but the basic method of inference is to begin with a goal proposition that we would like to prove, and then determine if we can prove that inference using our rules and beliefs. Some cognitive architectures, such as ICARUS (Langley & Choi, 2006) actually have the ability to do both types of reasoning.

In backward chaining, rules and beliefs take the same form as for forward chaining. Additionally, the goal is simply another logical proposition; as above, we'll assume the statements contain no logical connectives, although the reasoning system can be applied in more complicated cases. The system first checks if its goal is one of its given beliefs. If it is, then the goal has been proven to be true. Otherwise, it finds any rules that have the goal as a consequent. For example, if we had the rules $[a \rightarrow b, b \rightarrow c, d \rightarrow c]$, belief $a$, and goal $c$, the system would find the rules $[b \rightarrow c, d \rightarrow c]$. It then uses the antecedent (left side) of these rules as new goals. In this example, it's now trying to prove either $b$ or $d$. If repeats the process of checking if the (new) goal is one of its beliefs, and if not, finding rules that have the goal as a consequent. This process repeats until either there are no new rules to examine or a goal can be proven. In this example, the system would eventually have the goal of $a$ (since $a \rightarrow b$ and $b\rightarrow c$), and since this was a belief, it would recognize that it can prove the original goal $c$. 
<p>
(Citation for ICARUS paper: Langley, P., & Choi, D. (2006) A unified cognitive architecture for physical agents. *In Proceedings of AAAI*.)

<div class="alert alert-success" role="alert">
<h3>Part (a)</h3>
<p>
In `productionrules.py`, write a function `backward_chain(beliefs, rules, goal)` that returns true if the given goal (a string) can be proven with the given beliefs and rules. The same representation of beliefs and rules is used as in the previous problem. As in the previous problem, your function should be non-destructive. In the cell below write some test cases for your function. Make sure to test a variety of cases, including rule systems that may have cycles and cases where the goal cannot be proven as well as cases where the goal can be proven.

In [7]:
# Test the goal can be achieved
assert productionrules.backward_chain({"a"}, [("a","b"),
                                              ("b","c"), ("d","c")], "c") == True
# Test the goal cannot be achieved
assert productionrules.backward_chain({"a"}, [("a","f"),
                                              ("b","d"), ("c","d")], "c") == False
# Test the goal cannot be achieved and there exists a self-pointed belief
assert productionrules.backward_chain({"a"}, [("b","b"), ("b","c"),
                                             ("d","c")], "c") == False
# Test the goal can be achieved and there are mutiple beliefs pointing to the goal
assert productionrules.backward_chain({"a"}, [("d","c"),
                                              ("b","c"), ("c","d"),("a","d")], "c") == True
# Test the goal can be achieved and there exists a self-pointed belief
assert productionrules.backward_chain({"b"},[("d","c"), ("b","a"), ("c","d"),
                                             ("a","d"), ("a", "c"), ("b", "f")], "c") == True

### Comparing chaining systems

<div class="alert alert-success" role="alert">
<h3>Part (b)</h3>
<p>
Consider whether one of backward or forward chaining is always more efficient if the system has a single proposition that it would like to evaluate as supported (proven true) or not by the current beliefs and rules. Explain your answer.

 The backward chaining is more efficient. Both algorithms are in $O(n^2)$. However, for the backward chaining, the program will terminate when there exists a belief which proves the proposition. Whereas the forward chaining will keep running until the point that no rule is triggered. Then, it can judge whether the proposition is in the new beliefs. In the worst case, the backward chaining will have similar run time as the forward chaining if we just want to evaluate one proposition, while for most of the cases, the backward chaining will be faster. 

###  Uses for chaining systems

<div class="alert alert-success" role="alert">
<h3>Part (c)</h3>
<p>
Explain whether it seems like it would ever be useful for an agent to have both reasoning systems available. You can assume that the agent is acting in the environment for a prolonged period of time and it may gain information over time.

Both the forward chaining and backward chaining will benefit the agent. By applying the forward chaining, the agent will acquire a comprehensive knowledge about the environment based on the rule and agent's current beliefs. By applying those knowledge, the agent will obtain a big picture about what he/she is facing. This will help the agent with making future plan, and generate more knowledge over time. While for backward chaining, it will be useful when the agent has a target goal and just want to prove whether that goal is correct or not. This will help the agent when he/she is carrying out a specific tasks. 